In [48]:
import pandas as pd
import numpy as np
from datetime import datetime
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier, plot_importance
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [2]:
#Split the file since its too big for github, extract the file outside of .git repository
"""
hpg_res = pd.read_csv('C:\\Users\\mingt\\Downloads\\hpg_reserve.csv\\hpg_reserve.csv')
hpg_res.info()
df1 = hpg_res.iloc[0:100160]
df2 = hpg_res.iloc[100161:200320]
df1.to_csv('hpg_reserve1.csv')
df2.to_csv('hpg_reserve2.csv')
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000320 entries, 0 to 2000319
Data columns (total 4 columns):
hpg_store_id        object
visit_datetime      object
reserve_datetime    object
reserve_visitors    int64
dtypes: int64(1), object(3)
memory usage: 61.0+ MB


In [24]:
air_reserve = pd.read_csv('air_reserve.csv')
air_store_info= pd.read_csv('air_store_info.csv')
air_visit_data = pd.read_csv('air_visit_data.csv')
date_info = pd.read_csv('date_info.csv')
hpg_store_info = pd.read_csv('hpg_store_info.csv')
store_id_relation = pd.read_csv('store_id_relation.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [25]:
#concat files back after its been uploaded on github
hpg_reserve1 = pd.read_csv('hpg_reserve1.csv')
hpg_reserve2 = pd.read_csv('hpg_reserve2.csv')

hpg_reserve = pd.concat([hpg_reserve1, hpg_reserve2])
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])

In [26]:
air_combined = pd.merge(air_reserve, air_store_info, on='air_store_id', how='outer')
hpg_combined = pd.merge(hpg_reserve, hpg_store_info, on='hpg_store_id', how='left')

df = store_id_relation.merge(hpg_combined, on='hpg_store_id', how='left')
df2 = air_combined.merge(df, on='air_store_id', how='left')

### Primary key on column: 'air_store_id'

In [27]:
master_col = df2[['air_store_id']]
df2.head(5)

,air_store_id,visit_datetime_x,reserve_datetime_x,reserve_visitors_x,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_store_id,Unnamed: 0,visit_datetime_y,reserve_datetime_y,reserve_visitors_y,hpg_genre_name,hpg_area_name,latitude_y,longitude_y
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
2,air_877f79706adbfb06,2016-01-02 18:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3,air_877f79706adbfb06,2016-01-02 21:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
4,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-04 20:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN


#### 11171047 entries in 'air_store_id', with 829 unique restaurants. 

In [6]:
print(df2.info())
print(len(df2['air_store_id'].value_counts()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1171047 entries, 0 to 1171046
Data columns (total 17 columns):
air_store_id          1171047 non-null object
visit_datetime_x      1170265 non-null object
reserve_datetime_x    1170265 non-null object
reserve_visitors_x    1170265 non-null float64
air_genre_name        1171047 non-null object
air_area_name         1171047 non-null object
latitude_x            1171047 non-null float64
longitude_x           1171047 non-null float64
hpg_store_id          1114437 non-null object
Unnamed: 0            1112519 non-null float64
visit_datetime_y      1112519 non-null datetime64[ns]
reserve_datetime_y    1112519 non-null datetime64[ns]
reserve_visitors_y    1112519 non-null float64
hpg_genre_name        451960 non-null object
hpg_area_name         451960 non-null object
latitude_y            451960 non-null float64
longitude_y           451960 non-null float64
dtypes: datetime64[ns](2), float64(7), object(8)
memory usage: 160.8+ MB
None
829


### Add features to date_info

In [28]:
date_info['calendar_date'] = pd.to_datetime(date_info['calendar_date']).dt.date.astype(str)
#date_info['holiday_flg'] = date_info['holiday_flg'].map({1: 'Yes', 0: 'No'})
date_info['MTWTF'] = date_info['day_of_week'].map({'Monday': 1, 
                                                   'Tuesday': 2, 
                                                   'Wednesday': 3, 
                                                   'Thursday': 4, 
                                                   'Friday': 5,
                                                   'Saturday': 6, 
                                                   'Sunday': 7})

date_info['weekend_or_weekday'] = date_info['day_of_week'].map({'Monday': 0, 
                                                                'Tuesday': 0, 
                                                                'Wednesday': 0, 
                                                                'Thursday': 0, 
                                                                'Friday': 0,
                                                                'Saturday': 1, 
                                                                'Sunday': 1})


date_info2 = date_info.drop(['day_of_week'], axis=1)
date_info2.head(5)

,calendar_date,holiday_flg,MTWTF,weekend_or_weekday
0,2016-01-01,1,5,0
1,2016-01-02,1,6,1
2,2016-01-03,1,7,1
3,2016-01-04,0,1,0
4,2016-01-05,0,2,0


### Join sample_submission to date_info

In [29]:
sub_store = sample_submission['id'].apply(lambda x: str(x).split('_', 2)[:2])
sub_dates = pd.to_datetime(sample_submission['id'].apply(lambda x: str(x).split('_', 2)[2]).rename('Date'))
sub_stores = pd.Series(['_'.join(x) for x in sub_store]).rename('air_store_id')

sub_dt = pd.DataFrame({
        'air_store_id': sub_stores,
        'date': sub_dates.dt.date.astype(str),
        #'year': sub_dates.dt.year,
        'month': sub_dates.dt.month,
        'day': sub_dates.dt.day })

sub_df = pd.concat([sample_submission, sub_dt], axis=1)
sub_df2 = sub_df.merge(date_info2, left_on= sub_df['date'], right_on=date_info['calendar_date'])
sub_df3 = sub_df2.drop(['id', 'calendar_date', 'date'], axis =1)
sub_df3.head(5)

,visitors,air_store_id,day,month,holiday_flg,MTWTF,weekend_or_weekday
0,0,air_00a91d42b08b08d9,23,4,0,7,1
1,0,air_0164b9927d20bcc3,23,4,0,7,1
2,0,air_0241aa3964b7f861,23,4,0,7,1
3,0,air_0328696196e46f18,23,4,0,7,1
4,0,air_034a3d5b40d5b1b1,23,4,0,7,1


### Split datetime columns, and join with date_info

In [30]:
a = pd.to_datetime(df2['visit_datetime_x'])
#b = pd.to_datetime(df2['reserve_datetime_x'])
#c = pd.to_datetime(df2['visit_datetime_y'])
#d = pd.to_datetime(df2['reserve_datetime_y'])

datetime_df =pd.DataFrame({
        'air_visit_date': a.dt.date.astype(str),
        #'year': a.dt.year,
        'month': a.dt.month,
        'day': a.dt.day,
        
        #'air_visit_hour': a.dt.hour,
        #'air_visit_date': a.dt.date,
        #'air_visit_year': b.dt.year,
        
        #'air_reserve_month': b.dt.month,
        #'air_reserve_day': b.dt.day,
        #'air_reserve_hour': b.dt.hour,
         
        #'hpg_visit_date': c.dt.date,
        #'hpg_visit_year': c.dt.year,
        #'hpg_visit_month': c.dt.month,
        #'hpg_visit_day': c.dt.day,
        #'hpg_visit_hour': c.dt.hour,
        
        #'hpg_reserve_date': d.dt.date,
        #'hpg_reserve_year': d.dt.year,
        #'hpg_reserve_month': d.dt.month,
        #'hpg_reserve_day': d.dt.day,
        #'hpg_reserve_hour': d.dt.hour        
    }).fillna(0)

datetime_df2 = datetime_df.merge(date_info, left_on=datetime_df['air_visit_date'], 
                                 right_on=date_info['calendar_date'])

### Encode all categorical variable, then join with submission_file and training set

In [31]:
categorical_df = pd.concat([master_col, df2[['air_genre_name','air_area_name','latitude_x','longitude_x',
                      'hpg_genre_name','hpg_area_name']].fillna('None_Stated').apply(LabelEncoder().fit_transform)], axis=1)

#categorical_dummy = pd.get_dummies(df2[['air_genre_name','air_area_name','latitude','longitude', 'hpg_genre_name','hpg_area_name']].fillna('None_Stated'))

sub_df4 = sub_df3.merge(categorical_df, left_on=sub_df3['air_store_id'], right_on=categorical_df['air_store_id'])  

combined_visitors = pd.Series(df2['reserve_visitors_x'].fillna(0) + df2['reserve_visitors_y'].fillna(0)).rename('visitors')


train_df = pd.concat([datetime_df2, combined_visitors, categorical_df], axis=1)

### Encode all air_store_id together, drop unneeded columns

In [75]:
lbl= LabelEncoder()
ids = lbl.fit_transform(feats['air_store_id'].append(sub_df4['air_store_id_x']))
train_df['air_store_id_num'] = pd.Series(ids[:1171046])
sub_df4['air_store_id_num'] = pd.Series(ids[1171047:])

train_df2 = train_df.drop(['air_store_id', 'air_visit_date', 'calendar_date', 'day_of_week'], axis=1)
sub_df5 = sub_df4.drop(['air_store_id_x', 'air_store_id_y'], axis=1).reindex(columns=list(train_df2.columns.values))

In [33]:
train_df2.head(5)

,day,month,holiday_flg,MTWTF,weekend_or_weekday,visitors,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_genre_name,hpg_area_name,air_store_id_num
0,1.0,1.0,1.0,5.0,0.0,1.0,8,62,54,83,10,16,426.0
1,1.0,1.0,1.0,5.0,0.0,2.0,8,62,54,83,10,16,426.0
2,1.0,1.0,1.0,5.0,0.0,2.0,8,62,54,83,10,16,426.0
3,1.0,1.0,1.0,5.0,0.0,2.0,8,62,54,83,10,16,426.0
4,1.0,1.0,1.0,5.0,0.0,2.0,8,62,54,83,10,16,426.0


In [34]:
sub_df5.head(5)

,day,month,holiday_flg,MTWTF,weekend_or_weekday,visitors,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_genre_name,hpg_area_name,air_store_id_num
0,23,4,0,7,1,0,6,44,68,85,10,16,0
1,23,4,0,7,1,0,6,44,68,85,10,16,0
2,23,4,0,7,1,0,6,44,68,85,10,16,0
3,23,4,0,7,1,0,6,44,68,85,10,16,0
4,23,4,0,7,1,0,6,44,68,85,10,16,0


### Make some descriptive visualizations

In [58]:
vis_store = train_df2['visitors'].groupby(train_df2['air_store_id_num']).describe()
vis_dow = train_df2['visitors'].groupby(train_df2['MTWTF']).describe()
vis_store_dow = train_df2['visitors'].groupby([train_df2['air_store_id_num'], train_df2['MTWTF']]).describe()

In [73]:
vis_store_dow.head(20)

count       mean        std  min    25%   50%  \
air_store_id_num MTWTF                                                    
0.0              7.0       84.0   8.690476   5.053869  4.0   5.00   6.5   
1.0              4.0      142.0   3.316901   2.599426  1.0   2.00   2.0   
                 6.0       11.0   4.181818   2.926369  2.0   2.00   3.0   
4.0              7.0       14.0   6.642857  12.530622  2.0   2.25   3.0   
7.0              1.0      583.0   6.375643   6.064195  2.0   2.00   4.0   
                 6.0      684.0   6.554094   6.044370  2.0   2.00   4.0   
8.0              5.0       12.0  17.333333  11.547005  2.0  10.00  14.0   
10.0             1.0      224.0   6.267857   2.726657  3.0   4.00   6.0   
                 5.0     6121.0   7.206992   3.117333  3.0   5.00   6.0   
                 6.0     7971.0   7.926734   4.110415  3.0   5.00   7.0   
                 7.0    30989.0   7.480429   3.468521  3.0   5.00   6.0   
15.0             7.0     1746.0   9.875143   4.404529  4.0   6.00   9.0   
16.0             5.0        4.0   7.750000   2.061553  5.0   7.25   8.0   
21.0             4.0      464.0   7.133621   4.633831  4.0   4.00   6.0   
24.0             7.0        2.0   4.000000   2.828427  2.0   3.00   4.0   
25.0             5.0      466.0   8.070815   7.502926  4.0   4.00   5.0   
                 6.0     6374.0   9.492940   9.727394  3.0   4.00   5.0   
27.0             7.0    12064.0   9.078084   5.803480  3.0   5.00   7.0   
28.0             7.0      648.0  14.041667   7.041923  4.0   8.00  12.5   
35.0             1.0     8824.0   5.707729   2.801678  3.0   4.00   5.0   

                          75%   max  
air_store_id_num MTWTF               
0.0              7.0    11.00  23.0  
1.0              4.0     3.75  18.0  
                 6.0     5.00  10.0  
4.0              7.0     4.75  50.0  
7.0              1.0     8.00  42.0  
                 6.0     8.00  44.0  
8.0              5.0    25.25  37.0  
10.0             1.0     7.00  16.0  
                 5.0     8.00  24.0  
                 6.0     9.00  39.0  
                 7.0     9.00  31.0  
15.0             7.0    13.00  29.0  
16.0             5.0     8.50  10.0  
21.0             4.0     8.00  33.0  
24.0             7.0     5.00   6.0  
25.0             5.0     6.00  51.0  
                 6.0     9.00  69.0  
27.0             7.0    10.00  45.0  
28.0             7.0    18.00  41.0  
35.0             1.0     6.00  38.0

In [83]:
def rmsle(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y

    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())


X_train, X_test, y_train, y_test = train_test_split(train_df2, train_df2['visitors'], test_size=0.1, random_state=7)


xgbR = XGBRegressor(learning_rate=0.1,
                   objective='reg:linear')

xgbR.fit(X_train, y_train)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [84]:
gbr = GradientBoostingRegressor(learning_rate=0.1)
gbr.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [77]:



pred = xgbR.predict(X_test)
mse = rmsle(pred, y_test)
print(mse)

0.00111749550592


In [79]:
sub_feats = sub_df5.drop(['visitors'], axis=1)
#sub_pred = xgbR.predict()

In [82]:
pred2 = xgbR.predict(sub_df5)

OSError: [WinError -529697949] Windows Error 0xe06d7363

In [ ]:
['day', 'month', 'holiday_flg', 'MTWTF', 'weekend_or_weekday', 'visitors', 'air_genre_name', 'air_area_name', 'latitude_x', 'longitude_x', 'hpg_genre_name', 'hpg_area_name', 'air_store_id_num']
['day', 'month', 'holiday_flg', 'MTWTF', 'weekend_or_weekday', 'air_genre_name', 'air_area_name', 'latitude_x', 'longitude_x', 'hpg_genre_name', 'hpg_area_name', 'air_store_id_num']

vis_by_store = feats2['visitors'].groupby(feats2['air_store_id_num']).describe()
vis_by_dow = feats2['visitors'].groupby(feats2['MTWTF']).describe()